# GPU 설정

In [1]:
import tensorflow as tf

print(tf.__version__)

2.10.0


In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12357802847593552748
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5809889280
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8955454276431975758
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4080, pci bus id: 0000:07:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


# Import

In [5]:
import os
import zipfile
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from tqdm import tqdm

from PIL import Image, ImageDraw

In [6]:
# 각 supercategory별로 색상 지정
colors = {'roadlane': 'white', 
          'roadlane_ban': 'yellow', 
          'guidance': 'green', 
          'direction_ban': 'blue', 
          'speed_restriction': 'red',
          'protection_zone' : 'orange',
          'direction_guidance' : 'DeepSkyBlue',
          'stop_lane': 'brown',
          'crosswalk': 'cyan',
          'direction': 'pink',
          'etc': 'purple'}

# Image generate form Json

In [8]:
# 라벨링 데이터 가져오기(Json 파일)
zip_path_list = ['./01.데이터/2.Validation/라벨링데이터/VL1.온전.zip',
                 './01.데이터/2.Validation/라벨링데이터/VL2.비온전(물리).zip',
                 # './01.데이터/2.Validation/라벨링데이터/VL3.비온전(환경).zip',
                 #'./01.데이터/2.Validation/라벨링데이터/VL4.비온전(복합).zip'
                ]

# 마스킹 이미지 저장용 경로 생성
mask_all = './MASK_select/'

In [9]:
if not os.path.exists(mask_all):
    os.makedirs(mask_all)

### polyline + segmentation

In [10]:
for zip_path in tqdm(zip_path_list):
    # zip 파일 열기
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        # zip 파일 내부에 있는 파일 목록 가져오기
        file_list = zip_file.namelist()
        # 파일 목록 중 json 파일만 필터링
        json_file_list = [f for f in file_list if f.endswith('.json')]
        # json 파일 읽어오기
        for json_file_name in tqdm(json_file_list):
            filename = json_file_name[8:-5] # 파일 이름 지정
            with zip_file.open(json_file_name, 'r') as f:
                data = json.load(f)
            
            
            polylines = []
            segmeatations = []
            supercategories_poly = []
            supercategories_seg = []
            
            # annotation 정보에서 polyline 추출
            for ann in data['annotations']:
                try:
                    polyline = ann['polyline']
                    polylines.append(polyline)
                    category_id = ann['category_id']
                    supercategory = None
                    
                    for cat in data['categories']:
                        if cat['id'] == category_id:
                            supercategory = cat['supercategory']
                            break
                    supercategories_poly.append(supercategory)
                except KeyError:
                    pass

            # annotation 정보에서 segmentation 추출
            for ann in data['annotations']:
                try:
                    segmeatation = ann['segmentation']
                    segmeatations.append(segmeatation)
                    category_id = ann['category_id']
                    supercategory = None
                    for cat in data['categories']:
                        if cat['id'] == category_id:
                            supercategory = cat['supercategory']
                            break
                    supercategories_seg.append(supercategory)
                except KeyError:
                    pass
            
            #============================================================================
            # 검은색 배경 이미지 생성
            width = data['images'][0]['width']
            height = data['images'][0]['height']
            img = Image.new('RGB', (width, height), color='black')
            draw = ImageDraw.Draw(img)
            
            # polyline ================================================================
            for lines, supercat in zip(polylines, supercategories_poly):
                color = colors[supercat]
                for line in lines:
                    # line 리스트의 좌표를 튜플로 변환
                    points = [(line[i], line[i+1]) for i in range(0, len(line), 2)]
                    draw.line(points, fill=color, width=4)
                    
            # segmentation ============================================================
            # direction supercategory를 제외한 다른 supercategory 그리기
            for lines, supercat in zip(segmeatations, supercategories_seg):
                if supercat == 'direction':
                    continue  # direction은 나중에 그리기 위해 건너뜀
                color = colors[supercat]
                for line in lines:
                    # line 리스트의 좌표를 튜플로 변환
                    points = [(line[i], line[i+1]) for i in range(0, len(line), 2)]
                    draw.polygon(points, fill=color, outline=color)

            # direction supercategory 그리기
            for lines, supercat in zip(segmeatations, supercategories_seg):
                if supercat != 'direction':
                    continue  # direction만 그리기 위해 건너뜀
                color = colors[supercat]
                for line in lines:
                    # line 리스트의 좌표를 튜플로 변환
                    points = [(line[i], line[i+1]) for i in range(0, len(line), 2)]
                    draw.polygon(points, fill=color, outline=color)
    
            # 이미지 저장 ================================================================
            img.save(mask_all + filename + '_mask.png')

100%|██████████| 2/2 [14:47<00:00, 443.78s/it]


### extract_images_from_zip

In [11]:
import zipfile

def extract_images_from_zip(zip_path, folder_name):
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        file_list = zip_file.namelist()
        for file_name in tqdm(file_list):
            if folder_name in file_name:
                zip_file.extract(file_name, path='./')
                # print(f"Extracted {file_name}")

In [12]:
# 라벨링 데이터 가져오기(Json 파일)
zip_path_list = ['./01.데이터/2.Validation/원천데이터/VS1.온전.zip',
                 './01.데이터/2.Validation/원천데이터/VS2.비온전(물리).zip',
                 #'./01.데이터/2.Validation/원천데이터/VS3.비온전(환경).zip',
                 #'./01.데이터/2.Validation/원천데이터/VS4.비온전(복합).zip'
                 ]

for zip_path in zip_path_list:
    extract_images_from_zip(zip_path, 'IMAGE')

100%|██████████| 23112/23112 [01:47<00:00, 215.33it/s] 
